In [64]:
import base64
import requests
from requests_toolbelt import MultipartEncoder

This notebook shows some examples of how to call a model trained and deployed in the Peltarion platform from Python using requests with forms, mimicking a curl call.

The disadvantage of using this method is that you can only score one example at a time.

## Image classification

We will hide the deployment URL and the authentication token for security reasons. You can substitute the values found on the Deployment page for the model we want to use.

The following example assumes that you have a model trained on MNIST data (28x28 pixels, 3 color channels) and want to classify new images that the model has not seen.

In [20]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

We want to classify an image in file called `three.png` can be found in the `images` folder of this repo. The name of the only feature that this model uses is "Image". You will find the name of the feature(s) on the deployment page for your model.

Here the cURL call (which you can find a skeleton for on the deployment page) would look like this:

```
curl -X POST -F "Image=@images/three.png" \
-u " -- token from deployment page -- :" \
" -- URL from deployment page --"
```

Note the ":" that has to be present after the token. You can run curl from the notebook:

In [23]:
!curl -X POST -F "Image=@images/three.png" \
-u "-- insert your token -- :" \
"-- insert your URL --"

{"Number":{"0":1.1728967E-12,"1":1.904032E-23,"2":1.0426308E-9,"3":0.99999976,"4":8.7458367E-32,"5":2.2160066E-7,"6":1.1102806E-25,"7":8.750066E-19,"8":9.540412E-19,"9":6.100854E-9}}

But now let's do the same thing in Python. We start by defining a small function to get a prediction. The input_dict will be a dictionary containing information about your inputs.

In [24]:
def get_prediction(input_dict, token, url):
    payload = MultipartEncoder(input_dict)
    r = requests.post(url,
                      data=payload,
                      headers={'Content-Type': payload.content_type},
                      auth=(token, ''))
    return(eval(r.text))

We make a dictionary containing the name and file path of the image we want to classify. The name is again the feature name we see on the Deployment page. 

In [25]:
image, path = 'Image', 'images/three.png'
input_dict = {image: (path, open(path, 'rb'), 'image/png')}

Now we can use the function defined above by inserting the input_dict, the token and the URL.

In [19]:
pred = get_prediction(input_dict, token, url)
pred

{'Number': {'0': 1.1728967e-12,
  '1': 1.904032e-23,
  '2': 1.0426308e-09,
  '3': 0.99999976,
  '4': 8.7458367e-32,
  '5': 2.2160066e-07,
  '6': 1.1102806e-25,
  '7': 8.750066e-19,
  '8': 9.540412e-19,
  '9': 6.100854e-09}}

### Classifying several images

For classifying a batch of examples, it is better to use JSON requests (see the Peltarion_deployment_JSON_examples.ipynb notebook in this repo). 

## Using multiple inputs

Here we are using the California housing dataset together with images from Google maps, so that we have a set of numerical inputs as well as an image input.

In [48]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

For the numerical inputs, we just write key-value pairs consisting of the feature names from the deployment page and their corresponding values. For the image input, we add it to the dictionary as shown above.

In [49]:
img_path = 'images/15_5256_12656.png'
img_dict = {'image_path': (img_path, open(img_path, 'rb'), 'image/png')}
num_dict =  {
    "population": "1551.0",
    "totalBedrooms": "434.0",
    "totalRooms": "2202.0",
    "housingMedianAge": "52.0",
    "medianIncome": "3.12",
    "households": "514.0",
    "latitude": "37.88",
    "longitude": "-122.25"
}

input_dict = {**img_dict, **num_dict}

In [50]:
input_dict

{'image_path': ('images/15_5256_12656.png',
  <_io.BufferedReader name='images/15_5256_12656.png'>,
  'image/png'),
 'population': '1551.0',
 'totalBedrooms': '434.0',
 'totalRooms': '2202.0',
 'housingMedianAge': '52.0',
 'medianIncome': '3.12',
 'households': '514.0',
 'latitude': '37.88',
 'longitude': '-122.25'}

In [51]:
get_prediction(input_dict, token, url)

{'medianHouseValue': 204714.05}

## Image to image

Here we will send two images to a deployment and get two images back. The images come from the NoisyOffice dataset (https://archive.ics.uci.edu/ml/datasets/NoisyOffice) where the task is to clean images from stains and other imperfections.

The sole input feature is called `path_noisy` and the output is called `path_clean`.

In [59]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

In [60]:
image, path = 'path_noisy', 'images/FontLrm_Noisec_TE.png'
input_dict = {image: (path, open(path, 'rb'), 'image/png')}

The output will be returned as a base64 encoded string, which you can for example save to file.

In [61]:
output = get_prediction(input_dict, token, url)

In [67]:
decoded = base64.b64decode(output['path_clean'].split(',')[-1])
with open('images/reconstructed_image.png', 'bw') as outf:
    outf.write(decoded)